<a href="https://colab.research.google.com/github/JuliaFavaro/cmepda/blob/main/test_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pycuda

ModuleNotFoundError: ignored

In [3]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.4 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp310-cp310-linux_x86_64.whl size=661265 sha256=8c04a416250c4cfebb96f5ca87c0730ff50fabc4e717b42a63e10eafdc556051
  Stored in directory: /root/.cache/pip/wheels/1d/7b/06/82a395a243fce00035dea9914d92bbef0013401497d849f8bc
Successfully built pycuda


In [4]:
import pycuda

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [9]:
!nvidia-smi


Mon Oct 30 13:45:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
!ls

sample_data


In [11]:
!touch ciao

In [12]:
!rm ciao

In [15]:
from pycuda import autoinit
from pycuda.tools import DeviceData
specs = DeviceData()
print ('Max threads per block = '+str(specs.max_threads))
print ('Warp size ='+str(specs.warp_size))
print ('Warps per MP ='+str(specs.warps_per_mp))
print ('Thread Blocks per MP ='+str(specs.thread_blocks_per_mp))
print ('Registers ='+str(specs.registers))
print ('Shared memory ='+ str(specs.shared_memory))

Max threads per block = 1024
Warp size =32
Warps per MP =64
Thread Blocks per MP =8
Registers =65536
Shared memory =49152


In [59]:
%%writefile VecAdd.cu
# include <stdio.h>
# include <cuda_runtime.h>
// CUDA Kernel
__global__ void vectorAdd(const float *A, const float *B, float *C, int numElements)
{
int i = blockDim.x * blockIdx.x + threadIdx.x;
if (i<numElements){
  C[i] = A[i] + B[i];}
}
/**
* Host main routine
*/
int main(void)
{
int numElements = 15;
size_t size = numElements * sizeof(float);
printf("[Vector addition of %d elements]\n", numElements);
float a[numElements],b[numElements],c[numElements];
float *a_gpu,*b_gpu,*c_gpu;
cudaMalloc((void **)&a_gpu, size);
cudaMalloc((void **)&b_gpu, size);
cudaMalloc((void **)&c_gpu, size);
for (int i=0;i<numElements;++i ){
a[i] = i*i;
b[i] = i;
}
// Copy the host input vectors A and B in host memory to the device input vectors in device memory
printf("Copy input data from the host memory to the CUDA device\n");
cudaMemcpy(a_gpu, a, size, cudaMemcpyHostToDevice);
cudaMemcpy(b_gpu, b, size, cudaMemcpyHostToDevice);
// Launch the Vector Add CUDA Kernel
int threadsPerBlock = 256;
int blocksPerGrid =(numElements + threadsPerBlock - 1) / threadsPerBlock;
printf("CUDA kernel launch with %d blocks of %d threads\n", (blocksPerGrid threadsPerBlock));
vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(a_gpu, b_gpu, c_gpu,numElements);
// Copy the device result vector in device memory to the host result vector in host memory.
printf("Copy output data from the CUDA device to the host memory\n");
cudaMemcpy(c, c_gpu, size, cudaMemcpyDeviceToHost);
for (int i=0;i<numElements;++i ){
  printf("%f \n",c[i]);
}
// Free device global memory
cudaFree(a_gpu);
cudaFree(b_gpu);
cudaFree(c_gpu);
printf("Done\n");
return 0;
}

Overwriting VecAdd.cu


In [60]:
!nvcc -o VecAdd VecAdd.cu -arch=compute_35 -code=sm_35

nvcc warning : The 'compute_35', 'compute_37', 'sm_35', and 'sm_37' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
VecAdd.cu(34): error: expected a ")"

VecAdd.cu(34): warning #224-D: the format string requires additional arguments

1 error detected in the compilation of "VecAdd.cu".


NOW WE START WITH PYTHON

first we import the modules we need

In [31]:
from pycuda import autoinit

In [32]:
from pycuda import gpuarray
import numpy as np

In [33]:
aux = range(15)
a = np.array(aux).astype(np.float32)
b = (a*a).astype(np.float32)
c = np.zeros(len(aux)).astype(np.float32)

In [34]:
a_gpu = gpuarray.to_gpu(a)
b_gpu = gpuarray.to_gpu(b)
c_gpu = gpuarray.to_gpu(c)

In [35]:
c_gpu=a_gpu+b_gpu

In [36]:
c_gpu

array([  0.,   2.,   6.,  12.,  20.,  30.,  42.,  56.,  72.,  90., 110.,
       132., 156., 182., 210.], dtype=float32)

YOU CAN IMPLEMENT THE SAME OPERATION ON EACH ELEMENT OF THE VECTORS

In [42]:
from pycuda.elementwise import ElementwiseKernel
myCudaFunc = ElementwiseKernel(arguments = "float *a, float *b, float *c",
operation = "c[i] = a[i]+b[i]", #you can online write operations that are one line
name = "mySumK")

In [43]:
myCudaFunc(a_gpu,b_gpu,c_gpu)
c_gpu

array([  0.,   2.,   6.,  12.,  20.,  30.,  42.,  56.,  72.,  90., 110.,
       132., 156., 182., 210.], dtype=float32)

EVEN MORE GENERAL PROCEDURE

In [44]:
from pycuda.compiler import SourceModule

In [47]:
cudaCode = open("VecAdd.cu","r")
myCUDACode = cudaCode.read() #extraction of the code

In [49]:
myCode = SourceModule(myCUDACode) #compilation

CompileError: ignored

In [50]:
importedKernel = myCode.get_function("vectorAdd") #execution by importing it on Python

NameError: ignored

In [51]:
nThreadsPerBlock = 256
nBlockPerGrid = 1
nGridsPerBlock = 1

In [52]:
c_gpu.set(c)
c_gpu

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [53]:
importedKernel(a_gpu.gpudata, b_gpu.gpudata, c_gpu.gpudata,block=(nThreadsPerBlock,nBlockPerGrid,nGridsPerBlock))

NameError: ignored

In [54]:
c_gpu

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)